In [7]:
import stk, stko

import py3Dmol

from molecule import Molecule,show_stk_mol

from VisualizeTrajectory import TrajectoryVisualizer

In [2]:
smiles = 'O=C(O)c1ccccc1O'

In [3]:
acid = stk.BuildingBlock( smiles, [stk.CarboxylicAcidFactory()] )

show_stk_mol( acid )

stk.XyzWriter().write( acid, './acid.xyz' )

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
from ase.io import read, write

atoms = read( './acid.xyz' )

from tblite.ase import TBLite

atoms.calc = TBLite(method="GFN1-xTB")


from ase.optimize import BFGS

dynamics = BFGS( atoms, trajectory = 'trajectory.traj' )

dynamics.run( fmax=0.05 )

------------------------------------------------------------
  cycle        total energy    energy error   density error
------------------------------------------------------------
      1     -31.65638187411  -3.2008986E+01   7.4871983E-01
      2     -32.09408683081  -4.3770496E-01   4.0034996E-01
      3     -32.10035249059  -6.2656598E-03   1.6063599E-01
      4     -32.10499487871  -4.6423881E-03   3.5822468E-02
      5     -32.10860961966  -3.6147410E-03   7.5297335E-03
      6     -32.10880503775  -1.9541809E-04   3.0255356E-03
      7     -32.10883581667  -3.0778917E-05   4.1671763E-04
      8     -32.10883625935  -4.4268462E-07   1.5416671E-04
      9     -32.10883630807  -4.8717837E-08   5.0575687E-05
     10     -32.10883631071  -2.6456775E-09   1.4813212E-05
------------------------------------------------------------

 total:                                   1.593 sec
      Step     Time          Energy         fmax
BFGS:    0 18:35:22     -873.725940        4.4571
-----

True

In [5]:
from ase.io.trajectory import Trajectory

trajectory = Trajectory('./trajectory.traj')

molecules = []

for atoms in trajectory:

    write( 'intermediary.pdb', atoms )

    with open( 'intermediary.pdb' ) as ifile: molecule = Molecule(ifile)

    molecules.append( molecule )

In [10]:
show_trajectory = TrajectoryVisualizer(molecules)
show_trajectory.show_trajectory()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol